# Chess-Bot

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment

In [2]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn
Dataset pkl files found at /home/wumphlett/personal/Chess-Bot/data
Dataset val pkl files found at /home/wumphlett/personal/Chess-Bot/data
Done


## Pos2Vec

In [3]:
x_train, x_val = cb.dataset.pos2vec_dataset()

In [4]:
pos2vec = cb.model.train_pos2vec(x_train, x_val)

Training layer 1/4
Epoch 1/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.2070 - val_loss: 0.0970 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0934 - val_loss: 0.0901 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0871 - val_loss: 0.0845 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0827 - val_loss: 0.0812 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0801 - val_loss: 0.0791 - lr: 0.0046
Epoch 6/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0782 - val_loss: 0.0773 - lr: 0.0045
Epoch 7/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0765 - val_loss: 0.0756 - lr: 0.0044
Epoch 8/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0748 - val_loss: 0.0740 - lr: 0.0043
Epoch 9/200
40000/40000 [====

40000/40000 [==============================] - 76s 2ms/step - loss: 0.0406 - val_loss: 0.0405 - lr: 0.0013
Epoch 70/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0404 - val_loss: 0.0404 - lr: 0.0012
Epoch 71/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0403 - val_loss: 0.0402 - lr: 0.0012
Epoch 72/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0402 - val_loss: 0.0401 - lr: 0.0012
Epoch 73/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0400 - val_loss: 0.0400 - lr: 0.0012
Epoch 74/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0399 - val_loss: 0.0399 - lr: 0.0011
Epoch 75/200
40000/40000 [==============================] - 78s 2ms/step - loss: 0.0398 - val_loss: 0.0397 - lr: 0.0011
Epoch 76/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0397 - val_loss: 0.0396 - lr: 0.0011
Epoch 77/200
40000/40000 [===========================

40000/40000 [==============================] - 76s 2ms/step - loss: 0.0362 - val_loss: 0.0362 - lr: 3.3363e-04
Epoch 136/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0361 - val_loss: 0.0361 - lr: 3.2696e-04
Epoch 137/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0361 - val_loss: 0.0361 - lr: 3.2042e-04
Epoch 138/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0361 - val_loss: 0.0361 - lr: 3.1401e-04
Epoch 139/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0361 - val_loss: 0.0361 - lr: 3.0773e-04
Epoch 140/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0360 - val_loss: 0.0360 - lr: 3.0157e-04
Epoch 141/200
40000/40000 [==============================] - 77s 2ms/step - loss: 0.0360 - val_loss: 0.0360 - lr: 2.9554e-04
Epoch 142/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0360 - val_loss: 0.0360 - lr: 2.8963e-04
Epoch 143/200


40000/40000 [==============================] - 75s 2ms/step - loss: 0.0354 - val_loss: 0.0354 - lr: 8.9734e-05
Training layer 2/4
Epoch 1/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0705 - val_loss: 0.0593 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0553 - val_loss: 0.0522 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0502 - val_loss: 0.0486 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.0473 - val_loss: 0.0462 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 76s 2ms/step - loss: 0.0454 - val_loss: 0.0446 - lr: 0.0046
Epoch 6/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0439 - val_loss: 0.0433 - lr: 0.0045
Epoch 7/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0428 - val_loss: 0.0423 - lr: 0.0044
Epoch 8/200
40000/40000 [============

40000/40000 [==============================] - 79s 2ms/step - loss: 0.0324 - val_loss: 0.0323 - lr: 0.0013
Epoch 69/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0323 - val_loss: 0.0323 - lr: 0.0013
Epoch 70/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0323 - val_loss: 0.0323 - lr: 0.0012
Epoch 71/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0322 - val_loss: 0.0322 - lr: 0.0012
Epoch 72/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0322 - val_loss: 0.0322 - lr: 0.0012
Epoch 73/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.0322 - val_loss: 0.0322 - lr: 0.0012
Epoch 74/200
40000/40000 [==============================] - 74s 2ms/step - loss: 0.0322 - val_loss: 0.0321 - lr: 0.0011
Epoch 75/200
40000/40000 [==============================] - 75s 2ms/step - loss: 0.0321 - val_loss: 0.0321 - lr: 0.0011
Epoch 76/200
40000/40000 [===========================

...

40000/40000 [==============================] - 78s 2ms/step - loss: 0.0305 - val_loss: 0.0305 - lr: 3.4739e-04
Epoch 134/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0305 - val_loss: 0.0305 - lr: 3.4044e-04
Epoch 135/200
40000/40000 [==============================] - 80s 2ms/step - loss: 0.0305 - val_loss: 0.0304 - lr: 3.3363e-04
Epoch 136/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0304 - val_loss: 0.0304 - lr: 3.2696e-04
Epoch 137/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0304 - val_loss: 0.0304 - lr: 3.2042e-04
Epoch 138/200
40000/40000 [==============================] - 81s 2ms/step - loss: 0.0304 - val_loss: 0.0304 - lr: 3.1401e-04
Epoch 139/200
40000/40000 [==============================] - 80s 2ms/step - loss: 0.0304 - val_loss: 0.0304 - lr: 3.0773e-04
Epoch 140/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0304 - val_loss: 0.0304 - lr: 3.0157e-04
Epoch 141/200


40000/40000 [==============================] - 79s 2ms/step - loss: 0.0303 - val_loss: 0.0303 - lr: 9.3434e-05
Epoch 199/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0303 - val_loss: 0.0303 - lr: 9.1566e-05
Epoch 200/200
40000/40000 [==============================] - 79s 2ms/step - loss: 0.0303 - val_loss: 0.0303 - lr: 8.9734e-05
Training layer 4/4
Epoch 1/200
40000/40000 [==============================] - 84s 2ms/step - loss: 0.0792 - val_loss: 0.0611 - lr: 0.0050
Epoch 2/200
40000/40000 [==============================] - 83s 2ms/step - loss: 0.0543 - val_loss: 0.0491 - lr: 0.0049
Epoch 3/200
40000/40000 [==============================] - 84s 2ms/step - loss: 0.0459 - val_loss: 0.0433 - lr: 0.0048
Epoch 4/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0416 - val_loss: 0.0402 - lr: 0.0047
Epoch 5/200
40000/40000 [==============================] - 83s 2ms/step - loss: 0.0392 - val_loss: 0.0383 - lr: 0.0046
Epoch 6/200
40000/40000 [

40000/40000 [==============================] - 83s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0013
Epoch 67/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0013
Epoch 68/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0013
Epoch 69/200
40000/40000 [==============================] - 83s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0013
Epoch 70/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0012
Epoch 71/200
40000/40000 [==============================] - 81s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0012
Epoch 72/200
40000/40000 [==============================] - 84s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0012
Epoch 73/200
40000/40000 [==============================] - 83s 2ms/step - loss: 0.0312 - val_loss: 0.0312 - lr: 0.0012
Epoch 74/200
40000/40000 [===========================

40000/40000 [==============================] - 83s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.5448e-04
Epoch 133/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.4739e-04
Epoch 134/200
40000/40000 [==============================] - 85s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.4044e-04
Epoch 135/200
40000/40000 [==============================] - 84s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.3363e-04
Epoch 136/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.2696e-04
Epoch 137/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.2042e-04
Epoch 138/200
40000/40000 [==============================] - 83s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.1401e-04
Epoch 139/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 3.0773e-04
Epoch 140/200


40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 9.5341e-05
Epoch 198/200
40000/40000 [==============================] - 84s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 9.3434e-05
Epoch 199/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 9.1566e-05
Epoch 200/200
40000/40000 [==============================] - 82s 2ms/step - loss: 0.0309 - val_loss: 0.0309 - lr: 8.9734e-05


In [5]:
import numpy as np

ex = np.reshape(x_train[0][0][0], (1, 773))

print("ex")
print(ex)

print("enc")
print(pos2vec(ex)[0])

ex
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False  True False False False False False False
  False False  True False False False False False False False False False
   True False False False False False False False False False False False
  False False False False False False False False  True False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
   True False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fal

## Sandbox